# Module 1: Is This Even an ML Problem?

**Goal:** Learn to frame ML problems correctly and avoid the most common mistake (data leakage)

**Time:** ~20 minutes

**What you'll do:**
1. Explore the StreamCart dataset
2. Practice the 7-line framing template
3. Identify data leakage in features
4. Define a proper churn label

---

## Setup

Run this cell to load the data. No installation needed—just pandas and numpy.

In [ ]:
import pandas as pd
import numpy as np

# Load StreamCart customer data
df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/data/streamcart_customers.csv')

print(f"Loaded {len(df):,} customers")
print(f"Columns: {len(df.columns)}")
df.head()

## Part 1: Explore the Data

Before building any model, understand what you're working with.

In [ ]:
# What's in this dataset?
print("=== Column Types ===")
print(df.dtypes)
print("\n=== Basic Stats ===")
df.describe()

In [ ]:
# The target variable: churn_30d
print("=== Churn Distribution ===")
print(df['churn_30d'].value_counts())
print(f"\nChurn rate: {df['churn_30d'].mean():.1%}")

### Quick Check

**Question:** About what percentage of customers churned in the next 30 days?

This is your **baseline**. A model that predicts "no churn" for everyone would be right ~89% of the time. But that's useless for the business.

---

## Part 2: The 7-Line Framing Template

Before writing any model code, fill out this template. If you can't, you're not ready to build.

| Line | Question | Your Answer |
|------|----------|-------------|
| 1. Problem | What business outcome are we trying to improve? | |
| 2. Action | What will we DO with the prediction? | |
| 3. Prediction | What exactly does the model output? | |
| 4. Label | How do we define this in historical data? | |
| 5. Features | What info is available at prediction time? | |
| 6. Metric | How do we measure if the model helps? | |
| 7. Constraints | What limits exist in production? | |

In [ ]:
# TODO: Fill out the framing for StreamCart's churn problem
#
# Context: StreamCart's retention team can call 500 customers per week.
# They want to prioritize customers most likely to cancel.

problem_framing = {
    "problem": "????",      # What business metric are we improving?
    "action": "????",       # What will the retention team DO?
    "prediction": "????",   # What does the model output? (probability? score? yes/no?)
    "label": "????",        # How is churn defined in the data?
    "features": "????",     # List 3-5 features that would be available
    "metric": "????",       # How do we know if the model is good? (hint: 500/week capacity)
    "constraints": "????"   # Any production limits?
}

# Uncomment to see your answers:
# for k, v in problem_framing.items():
#     print(f"{k}: {v}")

In [ ]:
# ============================================
# SELF-CHECK: Run this to validate your framing
# ============================================

def check_framing(framing):
    errors = []
    
    # Check problem
    if 'churn' not in framing['problem'].lower() and 'cancel' not in framing['problem'].lower() and 'retain' not in framing['problem'].lower():
        errors.append("Problem should mention churn, cancellation, or retention")
    
    # Check action
    if framing['action'] == "????" or len(framing['action']) < 10:
        errors.append("Action should describe what the team will DO with predictions")
    
    # Check metric mentions capacity
    if '500' not in framing['metric'] and 'precision' not in framing['metric'].lower() and 'top' not in framing['metric'].lower():
        errors.append("Metric should account for the 500/week capacity (hint: precision@500)")
    
    # Check features don't include leakage
    leaky_terms = ['cancel_reason', 'churn_date', 'cancel_date']
    for term in leaky_terms:
        if term in framing['features'].lower():
            errors.append(f"'{term}' is leakage! It only exists AFTER someone churns.")
    
    if errors:
        print("❌ Issues found:")
        for e in errors:
            print(f"   - {e}")
    else:
        print("✓ Framing looks good!")
    
    return len(errors) == 0

check_framing(problem_framing)

---

## Part 3: Spotting Data Leakage

This is the **#1 mistake** in applied ML. Leakage means using information that wouldn't be available at prediction time.

In [ ]:
# Look at all columns
print("=== All Columns ===")
for col in df.columns:
    print(f"  {col}")

In [ ]:
# TODO: Which columns would cause DATA LEAKAGE if used as features?
#
# Remember: At prediction time (snapshot_date), we're predicting if they'll
# churn in the NEXT 30 days. Any info that only exists AFTER they churn is leakage.

leaky_columns = [
    # "???",  # List columns that would leak
    # "???",
]

safe_columns = [
    # "???",  # List columns that are safe to use
    # "???",
]

In [ ]:
# ============================================
# SELF-CHECK: Verify your leakage detection
# ============================================

KNOWN_LEAKY = {'churn_30d', 'churn_date', 'cancel_reason'}
KNOWN_SAFE = {'tenure_months', 'logins_last_7d', 'logins_last_30d', 
              'support_tickets_last_30d', 'items_skipped_last_3_boxes',
              'nps_score', 'plan_type', 'avg_order_value'}

your_leaky = set(leaky_columns)
your_safe = set(safe_columns)

# Check leaky
if KNOWN_LEAKY.issubset(your_leaky):
    print("✓ Correctly identified leaky columns!")
else:
    missing = KNOWN_LEAKY - your_leaky
    print(f"❌ Missed leaky columns: {missing}")
    print("   Hint: These only exist AFTER someone churns")

# Check safe
if len(your_safe) >= 5:
    print(f"✓ Identified {len(your_safe)} safe features")
else:
    print(f"⚠️  Only {len(your_safe)} safe features. Look for behavioral features.")

---

## Part 4: Why Leakage Is Dangerous

Let's see what happens if you accidentally use a leaky feature.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Split data
X = df[['tenure_months', 'logins_last_30d', 'support_tickets_last_30d']].fillna(0)
y = df['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model (no leakage)
model_clean = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
model_clean.fit(X_train, y_train)

clean_auc = roc_auc_score(y_test, model_clean.predict_proba(X_test)[:, 1])
print(f"AUC without leakage: {clean_auc:.3f}")
print("This is realistic performance.")

In [ ]:
# Now let's "accidentally" include a leaky feature
# cancel_reason is encoded (it only exists for churners!)

df['has_cancel_reason'] = df['cancel_reason'].notna().astype(int)

X_leaky = df[['tenure_months', 'logins_last_30d', 'support_tickets_last_30d', 'has_cancel_reason']].fillna(0)

X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_leaky, y, test_size=0.2, random_state=42)

model_leaky = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
model_leaky.fit(X_train_l, y_train_l)

leaky_auc = roc_auc_score(y_test_l, model_leaky.predict_proba(X_test_l)[:, 1])
print(f"AUC with leakage: {leaky_auc:.3f}")
print("\n⚠️  This is suspiciously perfect! The model is 'cheating'.")
print("   In production, has_cancel_reason would always be 0 (they haven't canceled yet).")

### The Lesson

If your AUC is above 0.95, **be suspicious**. Real-world churn models typically achieve 0.70-0.85.

A model with leakage will:
- Look amazing in development
- Fail completely in production
- Waste months of work

---

## Part 5: Defining the Label Properly

The label `churn_30d` is already defined. But let's understand what goes into defining it correctly.

In [ ]:
# What's in our label?
print("snapshot_date:", df['snapshot_date'].iloc[0])
print("\nFor churners:")
churners = df[df['churn_30d'] == 1][['customer_id', 'snapshot_date', 'churn_date']].head(5)
print(churners)

print("\nFor non-churners:")
stayers = df[df['churn_30d'] == 0][['customer_id', 'snapshot_date', 'churn_date']].head(5)
print(stayers)

In [ ]:
# TODO: If you had to define churn_30d from raw data, how would you do it?
#
# Complete this function:

def define_churn_label(df, snapshot_date, window_days=30):
    """
    Define churn label: Did the customer cancel within window_days after snapshot_date?
    
    Parameters:
    -----------
    df : DataFrame with 'cancel_date' column (or None if still active)
    snapshot_date : The point-in-time we're predicting FROM
    window_days : How many days to look forward
    
    Returns:
    --------
    Series with 1 (churned) or 0 (stayed)
    """
    # TODO: Implement this
    # Hint: churn = 1 if cancel_date is between snapshot_date and snapshot_date + window_days
    
    pass  # Replace with your implementation

# Test your function:
# my_labels = define_churn_label(df, '2024-01-15', 30)
# print(f"My label matches: {(my_labels == df['churn_30d']).mean():.1%}")

---

## 📝 Final Exercise: Explain It

Your VP asks: "Why can't we just predict churn using their subscription status?"

Write a 3-4 sentence response explaining why that's circular reasoning.

In [ ]:
# Write your response here (as a comment or string):

vp_response = """
YOUR RESPONSE HERE
"""

print(vp_response)

---

## ✅ Module 1 Complete!

**What you learned:**
- The 7-line framing template
- How to identify data leakage
- Why leakage creates fake performance
- How to define labels properly

**Key takeaway:** The most important ML skill isn't coding—it's asking "Would I have this data at prediction time?"

**Next:** [Module 2: Your First Prediction Model →](./module_02_logistic_regression.ipynb)